# 1. Coleta de dados
- Por meio de scraping do sítio virtual da ANBIMA, entre os resultados de pesquisa de fundos ESG.
- Faz primeiro o scraping dos links das páginas de resultados de pesquisa. Em seguida, acessa cada um dos links, faz o scraping da página respectiva e descarrega o regulamento. Trata os erros numa segunda tentativa, ressaltando a necessidade de o usuário descarregar os regulamentos manualmente. Finalmente, corrige erros de formatação.
    - Requer instalação de webdriver Chromedriver na variável de ambiente PATH. Mais informações em README.md.
- Converte arquivos pdf em txt e salva na pasta ..data/txt.

## Scraping

### Elabora lista de todos os links

In [1]:
import sys
sys.path.append('../src')
import os
import scraping
links = []
erros = []
encerrados = []
metadados = []
diretorio = '../data/pdf/'
if not os.path.exists(diretorio):
    # Se não existir, cria o diretório
    os.makedirs(diretorio)

In [2]:
links = scraping.buscar_links('https://data.anbima.com.br/busca/fundos/?q=&page=1&size=100&asg=Sim') # Primeira página de pesquisa - baseada no filtro ASG da Anbima
links2 = scraping.buscar_links('https://data.anbima.com.br/busca/fundos/?q=%22esg%22%20&page=1&size=100&') # Segunda página de pesquisa - baseada na palavra-chave ESG, independente de filtro Anbima
links3 = scraping.buscar_links('https://data.anbima.com.br/busca/fundos/?q=%22esg%22%20&page=2&size=100&') # Terceira página de pesquisa - baseada na palavra-chave ESG, independente de filtro Anbima
links4 = scraping.buscar_links('https://data.anbima.com.br/busca/fundos/?q=%22asg%22%20&page=1&size=50&') # Quarta página de pesquisa - baseada na palavra-chave ASG, independente de filtro Anbima
links5 = scraping.buscar_links('https://data.anbima.com.br/busca/fundos/?q=%22is%22&page=1&size=100&') # Quinta página de pesquisa - baseada na palavra-chave IS, independente de filtro Anbima


total_links = links + links2 + links3 + links4 + links5
total_links = list(set(total_links)) # Eliminar links duplicados
print(f"Total de links pesquisados: {len(total_links)}")

Total de links pesquisados: 197


### Descarrega regulamentos

In [3]:
for indice, link in enumerate(total_links):
    print(f'Processando {indice + 1}/{len(total_links)}. Faltam {len(total_links) - indice - 1}.')
    try:
        # Scraping dos metadados
        data = scraping.scraping_metadados(link)
        if data == False:
            print('Fundo encerrado.')
            encerrados.append(((link, f"Fundo encerrado. Scraping descartado.")))
            continue
        else:
            metadados.append(data)
            safe_name = data["Arquivo"]
            file_path = os.path.join(diretorio, f'{safe_name}.pdf')
            # Verifica se o arquivo já existe antes de tentar baixá-lo
            if not os.path.exists(file_path):
                download_result = scraping.download_regulamento(safe_name, diretorio)
                if not download_result:
                    erros.append((link, "Falha no download. Arquivo não é PDF. Baixar manualmente. Fundo já registrado nos metadados."))
            else:
                print(f'O arquivo {safe_name}.pdf já existe. Download desnecessário.')     
    except Exception as e:
        print(f"Erro inesperado ao processar {link}: {e}")
        erros.append((link, f"Erro inesperado: {e}. Fundo não registrado nos metadados!"))
        
print(f"Processamento concluído. Total de erros: {len(erros)}")
if erros:
    print("Links com erro:")
    for erro in erros:
        print(erro)


    

Processando 1/197. Faltam 196.
O arquivo Aviva IGC ESG IA FC FI Mult Cred Priv IE.pdf já existe. Download desnecessário.
Processando 2/197. Faltam 195.
Fundo encerrado.
Processando 3/197. Faltam 194.
Fundo encerrado.
Processando 4/197. Faltam 193.
O arquivo BNPP M.CRED FI RF CRED PRIV LP SUST IS.pdf já existe. Download desnecessário.
Processando 5/197. Faltam 192.
Fundo encerrado.
Processando 6/197. Faltam 191.
O arquivo ITAÚ INDEX ESG ÁGUA AÇÕES FX IE FC DE IS.pdf já existe. Download desnecessário.
Processando 7/197. Faltam 190.
O arquivo SCHRODER BEST IDEAS ESG FIA IS.pdf já existe. Download desnecessário.
Processando 8/197. Faltam 189.
O arquivo SCH SUST AÇÕES GLOB USD FC FIA IS IE.pdf já existe. Download desnecessário.
Processando 9/197. Faltam 188.
O arquivo BB AÇÕES EQUIDADE IS FI.pdf já existe. Download desnecessário.
Processando 10/197. Faltam 187.
Fundo encerrado.
Processando 11/197. Faltam 186.
Fundo encerrado.
Processando 12/197. Faltam 185.
O arquivo BRA P ES EQ GLO FC FI M

- observação: descarregar regulamentos listados em erros manualmente mediante pesquisa na internet

In [4]:
len(metadados) +  len(encerrados) == len(total_links)# Confere se total de metadados corresponde a todos os links pesquisados

True

## Corrige problemas no arquivo de metadados
- O arquivo de metadados tem alguns problemas.
  1. Em alguns resultados, o botão "Baixar" ficou assinalado como se fosse um valor da tabela de dados disponibilizada pela Anbima. Ele ocupa valores deixados em branco.
  2. O campo "Última atualização do regulamento" teve o scraping truncado e figura como 'Regulamento\nData de atualização indisponível': 'Baixar'.
  3. Foi incluído o campo em branco '': ''.
  4. O valor da cota e a rentabilidade do fundo no ano foram incluídos como valores de um mesmo campo.

In [5]:
# Corrige problemas iniciais
scraping.problema_1(metadados)
scraping.problema_2(metadados)
scraping.problema_3(metadados)
scraping.problema_4(metadados)

In [6]:
import random
random.sample(metadados, 15) # Confere integridade da lista

[{'CNPJ': '39.490.897/0001-14',
  'Código ANBIMA': '568295',
  'ISIN': 'BR065RCTF004',
  'Patrimônio Líquido': 'R$ 12.277.523,16',
  'Taxa de administração ao ano': '0,80%',
  'Primeiro aporte': '09/12/2020',
  'Classe ANBIMA': 'Ações',
  'Tipo ANBIMA': 'Ações Invest. no Exterior',
  'Classe CVM': 'Ações',
  'Sub Classe CVM': 'Invest. no Exterior',
  'Fundo ESG': 'Sim',
  'Composição do fundo': 'FIC',
  'Fundo Master': 'Não',
  'Estrutura de gestão': 'Feeder',
  'Benchmark': 'Não se Aplica',
  'Foco de atuação': 'Não se aplica',
  'Investimento no exterior': '> 67%',
  'Credito privado': 'Não',
  'Tipo de investidor': 'Não Há Restrição',
  'Característica do investidor': 'Inv. Qualificado',
  'Cota de abertura': 'Não',
  'Fundo permite alavancagem': 'Não',
  'Aberto estatutariamente': 'Sim',
  'Tributação Alvo': 'Renda Variável',
  'Permite aplicação automática': 'Não',
  'Moeda': 'R$',
  'Administrador': 'ITAU UNIBANCO SA',
  'Gestor': 'ITAU UNIBANCO ASSET MANAGEMENT LTDA',
  'Distrib

In [7]:
## Salva metadados em arquivo json. 
import json
with open('../data/json/metadados.json', 'w', encoding='utf-8') as f:
        json.dump(metadados, f, ensure_ascii=False, indent=4)